In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import json
import matplotlib

In [2]:
matplotlib.use('Agg')

In [3]:
class TemporalConvNet(nn.Module):
    def __init__(self, input_size, output_size=1, num_channels=[32, 32, 16], kernel_size=3, dropout=0.1):
        super(TemporalConvNet, self).__init__()
        
        layers = []
        num_levels = len(num_channels)
        
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = input_size if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            
            layers.extend([
                nn.Conv1d(in_channels, out_channels, kernel_size,
                         stride=1, dilation=dilation_size,
                         padding=(kernel_size-1) * dilation_size),
                nn.BatchNorm1d(out_channels),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
        
        self.network = nn.Sequential(*layers)
        self.output_layer = nn.Linear(num_channels[-1], output_size)
        
    def forward(self, x):
        out = self.network(x)
        out = out[:, :, -1]
        return self.output_layer(out)

In [4]:
def load_world_bank_data():
    print("Loading World Bank Development Indicators...")
    data_path = "/Users/shahbaz/Desktop/learning/msds/spring-2025/ml-for-ds/project/world_bank_common_indicators.csv"
    df = pd.read_csv(data_path)
    
    cols = [col for col in df.columns if '[YR' in col and col != 'Series Code']
    
    melted_df = pd.melt(df, 
        id_vars=['Country Name', 'Series Name'],
        value_vars=cols,
        var_name='Year', 
        value_name='Value')
    
    melted_df['Year'] = melted_df['Year'].str.extract('(\d{4})').astype(int)
    
    melted_df = melted_df.dropna(subset=['Value'])
    
    print(f"Data loaded: {len(melted_df)} records")
    print(f"Year range: {melted_df['Year'].min()} - {melted_df['Year'].max()}")
    print(f"Countries: {melted_df['Country Name'].nunique()}")
    print(f"Indicators: {melted_df['Series Name'].nunique()}")
    
    return melted_df

In [5]:
def prepare_country_indicator_data(df, country_name, indicator_name):
    country_data = df[
        (df['Country Name'] == country_name) & 
        (df['Series Name'] == indicator_name)
    ].sort_values('Year')
    
    if len(country_data) < 20:
        raise ValueError(f"Insufficient data for {country_name} - {indicator_name}")
    
    return country_data

In [6]:
def create_sequences(data, sequence_length=8):
    values = data['Value'].values
    years = data['Year'].values
    
    # Scaling the data
    scaler = MinMaxScaler()
    values_scaled = scaler.fit_transform(values.reshape(-1, 1)).flatten()
    
    # Creating sequences
    X, y = [], []
    for i in range(len(values_scaled) - sequence_length):
        X.append(values_scaled[i:i+sequence_length])
        y.append(values_scaled[i+sequence_length])
    
    return np.array(X), np.array(y), years, scaler

In [7]:
def split_train_test(X, y, years, split_year=2010):
    split_idx = None
    for i, year in enumerate(years):
        if year > split_year:
            split_idx = max(0, i - X.shape[1])  # Account for sequence length
            break
    
    if split_idx is None:
        split_idx = int(0.8 * len(X))  # Fallback to 80% split
    
    return X[:split_idx], X[split_idx:], y[:split_idx], y[split_idx:], split_idx

In [8]:
def train_tcn_model(X_train, y_train, input_size, epochs=150, lr=0.001):    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Creating model
    model = TemporalConvNet(input_size=1, num_channels=[32, 32, 16]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    
    # Converting to tensors
    X_train_tensor = torch.FloatTensor(X_train).unsqueeze(1).to(device)  # Add channel dim
    y_train_tensor = torch.FloatTensor(y_train).to(device)
    
    # Training loop
    model.train()
    losses = []
    
    print(f"Training TCN for {epochs} epochs")
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train_tensor).squeeze()
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        losses.append(loss.item())
        
        if (epoch + 1) % 30 == 0:
            print(f"  Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")
    
    print("Training completed!")
    return model, losses, device

In [9]:
def evaluate_model(model, X_test, y_test, device):
    model.eval()
    X_test_tensor = torch.FloatTensor(X_test).unsqueeze(1).to(device)
    with torch.no_grad():
        predictions = model(X_test_tensor).squeeze().cpu().numpy()
    
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)
    
    return predictions, {'mse': mse, 'rmse': rmse, 'r2': r2}

In [10]:
def create_visualization(y_train, y_test, train_pred, test_pred, years, split_idx, 
                        scaler, country, indicator, sequence_length):
    # Inverse transform for plotting
    y_train_orig = scaler.inverse_transform(y_train.reshape(-1, 1)).flatten()
    y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    train_pred_orig = scaler.inverse_transform(train_pred.reshape(-1, 1)).flatten()
    test_pred_orig = scaler.inverse_transform(test_pred.reshape(-1, 1)).flatten()
    
    # Get corresponding years
    years_adj = years[sequence_length:]
    years_train = years_adj[:len(y_train_orig)]
    years_test = years_adj[len(y_train_orig):len(y_train_orig)+len(y_test_orig)]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Plot 1: Complete time series
    axes[0,0].plot(years_train, y_train_orig, 'b-', label='Training Actual', alpha=0.7, linewidth=1.5)
    axes[0,0].plot(years_train, train_pred_orig, 'cyan', linestyle='--', label='Training Predicted', alpha=0.8)
    axes[0,0].plot(years_test, y_test_orig, 'red', label='Test Actual', linewidth=2.5, marker='o', markersize=4)
    axes[0,0].plot(years_test, test_pred_orig, 'orange', linestyle='--', label='Test Predicted', linewidth=2.5, marker='s', markersize=4)
    axes[0,0].axvline(x=2010, color='black', linestyle=':', linewidth=2, label='Train/Test Split')
    axes[0,0].set_title(f'TCN Model: {country} - {indicator}', fontsize=14, fontweight='bold')
    axes[0,0].set_xlabel('Year')
    axes[0,0].set_ylabel(indicator)
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # Plot 2: Test period focus
    if len(years_test) > 0:
        axes[0,1].plot(years_test, y_test_orig, 'ro-', label='Actual', markersize=6, linewidth=2)
        axes[0,1].plot(years_test, test_pred_orig, 'bs--', label='TCN Predicted', markersize=6, linewidth=2)
        r2_test = r2_score(y_test_orig, test_pred_orig)
        axes[0,1].set_title(f'Test Period (2011-2024)\nR² = {r2_test:.4f}', fontsize=12)
        axes[0,1].set_xlabel('Year')
        axes[0,1].set_ylabel(indicator)
        axes[0,1].legend()
        axes[0,1].grid(True, alpha=0.3)
    
    # Plot 3: Actual vs Predicted scatter
    if len(test_pred_orig) > 0:
        axes[1,0].scatter(y_test_orig, test_pred_orig, alpha=0.7, color='purple', s=50)
        min_val = min(y_test_orig.min(), test_pred_orig.min())
        max_val = max(y_test_orig.max(), test_pred_orig.max())
        axes[1,0].plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.8, linewidth=2)
        axes[1,0].set_title('Actual vs Predicted (Test Set)')
        axes[1,0].set_xlabel('Actual Values')
        axes[1,0].set_ylabel('Predicted Values')
        axes[1,0].grid(True, alpha=0.3)
    
    # Plot 4: Residuals
    if len(test_pred_orig) > 0:
        residuals = y_test_orig - test_pred_orig
        axes[1,1].scatter(range(len(residuals)), residuals, alpha=0.7, color='green')
        axes[1,1].axhline(y=0, color='red', linestyle='--', linewidth=2)
        axes[1,1].set_title('Prediction Residuals (Test Set)')
        axes[1,1].set_xlabel('Sample Index')
        axes[1,1].set_ylabel('Residual')
        axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save plot
    filename = f"tcn_prediction_{country.replace(' ', '_')}_{indicator.replace(' ', '_').replace('(', '').replace(')', '').replace(',', '')}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Visualization saved: {filename}")
    
    return filename

In [11]:
def main():
    print("=" * 60)
    print("WORLD BANK TIME SERIES PREDICTION WITH TCN")
    print("Algorithm: Temporal Convolutional Network (TCN)")
    print("Training: 1960-2010 | Testing: 2011-2024")
    print("=" * 60)
    
    try:
        df = load_world_bank_data()
        
        # Try different combinations
        indicators_to_try = [
            'GDP growth (annual %)',
            'GDP per capita growth (annual %)',
            'Life expectancy at birth, total (years)',
            'Fertility rate, total (births per woman)'
        ]
        print(f"Using indicators {indicators_to_try}")
        
        countries_to_try = [
            'United States', 'Germany', 'Japan', 'United Kingdom', 
            'France', 'Brazil', 'India', 'China'
        ]

        print(f"Using countries {countries_to_try}")
        
        best_data = None
        selected_country = None
        selected_indicator = None
        
        for indicator in indicators_to_try:
            for country in countries_to_try:
                try:
                    country_data = prepare_country_indicator_data(df, country, indicator)
                    if len(country_data) >= 40:
                        best_data = country_data
                        selected_country = country
                        selected_indicator = indicator
                        break
                except:
                    continue
            if best_data is not None:
                break
        
        if best_data is None:
            print("Could not find suitable data combination")
            return
        
        print(f"Selected: {selected_country} - {selected_indicator}")
        print(f"Data points: {len(best_data)}")
        
        # Prepare sequences
        sequence_length = 8
        X, y, years, scaler = create_sequences(best_data, sequence_length)
        print(f"Created {len(X)} sequences of length {sequence_length}")
        
        # Split data
        X_train, X_test, y_train, y_test, split_idx = split_train_test(X, y, years)
        print(f"Training samples: {len(X_train)}")
        print(f"Test samples: {len(X_test)}")
        
        # Train model
        model, train_losses, device = train_tcn_model(X_train, y_train, input_size=1)
        
        # Evaluate it
        model.eval()
        with torch.no_grad():
            X_train_tensor = torch.FloatTensor(X_train).unsqueeze(1).to(device)
            X_test_tensor = torch.FloatTensor(X_test).unsqueeze(1).to(device)
            
            train_pred = model(X_train_tensor).squeeze().cpu().numpy()
            test_pred = model(X_test_tensor).squeeze().cpu().numpy()
        
        # Evaluate
        _, test_metrics = evaluate_model(model, X_test, y_test, device)
        
        print(f"MODEL PERFORMANCE:")
        print(f"   Test MSE: {test_metrics['mse']:.6f}")
        print(f"   Test RMSE: {test_metrics['rmse']:.6f}")
        print(f"   Test R²: {test_metrics['r2']:.4f}")
        
        # Create visualization
        plot_filename = create_visualization(
            y_train, y_test, train_pred, test_pred, years, split_idx,
            scaler, selected_country, selected_indicator, sequence_length
        )
        
        # Save results
        results = {
            'algorithm': 'TCN (Temporal Convolutional Network)',
            'selection_reasoning': 'TCN chosen for long sequences, structural break robustness, and parallel training',
            'country': selected_country,
            'indicator': selected_indicator,
            'data_points': len(best_data),
            'sequence_length': sequence_length,
            'train_period': '1960-2010',
            'test_period': '2011-2024',
            'train_samples': len(X_train),
            'test_samples': len(X_test),
            'model_parameters': sum(p.numel() for p in model.parameters()),
            'performance_metrics': test_metrics,
            'visualization_file': plot_filename
        }
        
        with open('final_tcn_results.json', 'w') as f:
            json.dump(results, f, indent=2)
        
        print(f"ANALYSIS COMPLETE!")
        print(f"TCN Model R² Score: {test_metrics['r2']:.4f}")
        print(f"Results saved: final_tcn_results.json")
        print(f"Plot saved: {plot_filename}")
        
        return results
        
    except Exception as e:
        print(f"Exception during analysis: {e}")
        import traceback
        traceback.print_exc()
        return None

In [12]:
if __name__ == "__main__":
    results = main()

WORLD BANK TIME SERIES PREDICTION WITH TCN
Algorithm: Temporal Convolutional Network (TCN)
Training: 1960-2010 | Testing: 2011-2024
Loading World Bank Development Indicators...
Data loaded: 35957 records
Year range: 1960 - 2024
Countries: 31
Indicators: 20
Using indicators ['GDP growth (annual %)', 'GDP per capita growth (annual %)', 'Life expectancy at birth, total (years)', 'Fertility rate, total (births per woman)']
Using countries ['United States', 'Germany', 'Japan', 'United Kingdom', 'France', 'Brazil', 'India', 'China']
Selected: United States - GDP growth (annual %)
Data points: 63
Created 55 sequences of length 8
Training samples: 42
Test samples: 13
Using device: cpu
Training TCN for 150 epochs
  Epoch 30/150, Loss: 0.052304
  Epoch 60/150, Loss: 0.039851
  Epoch 90/150, Loss: 0.051889
  Epoch 120/150, Loss: 0.041352
  Epoch 150/150, Loss: 0.036601
Training completed!
MODEL PERFORMANCE:
   Test MSE: 0.027833
   Test RMSE: 0.166831
   Test R²: 0.0375
Visualization saved: tcn_p